In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
from google.colab import files
uploaded =files.upload()
# drive.mount('/content/drive')

Saving FakeNewsNet.csv to FakeNewsNet.csv


In [ ]:
df_news=pd.read_csv('FakeNewsNet.csv')

In [ ]:
df_news.shape

(23196, 5)

In [ ]:
df_news.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [ ]:
df=df_news

In [ ]:
df.isnull().sum()

,0
title,0
news_url,330
source_domain,330
tweet_num,0
real,0


In [ ]:
# dropping the null values
df=df.dropna()

In [ ]:
df.isnull().sum()

,0
title,0
news_url,0
source_domain,0
tweet_num,0
real,0


In [ ]:
df.shape

(22866, 5)

In [ ]:
# dropping the tweet number
df=df.drop('tweet_num',axis=1)

In [ ]:
df.describe()

,real
count,22866.000000
mean,0.759687
std,0.427283
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
df.shape

(22866, 4)

In [ ]:
df.head()

,title,news_url,source_domain,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,1


In [ ]:
X=df.drop(columns='real',axis=1)
Y=df['real']

In [ ]:
X

,title,news_url,source_domain
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com
...,...,...,...
23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk
23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com
23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com
23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com


In [ ]:
Y

,real
0,1
1,1
2,1
3,1
4,1
...,...
23191,1
23192,0
23193,1
23194,0


In [ ]:
# Performing the stemming in title
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content


In [ ]:
df['title']=df['title'].apply(stemming)

In [ ]:
df['title']

,title
0,kandi burruss explod rape accus real housew at...
1,peopl choic award best red carpet look
2,sophia bush send sweet birthday messag one tre...
3,colombian singer maluma spark rumour inappropr...
4,gossip girl year later upper east sider shock ...
...,...
23191,pippa middleton wed case miss pippa marri lace...
23192,zayn malik gigi hadid shock split chanc reunit...
23193,jessica chastain recal moment mother boyfriend...
23194,tristan thompson feel dump khlo kardashian ref...


In [ ]:
X=df.drop(columns='real',axis=1)
Y=df['real']

In [ ]:
X

,title,news_url,source_domain
0,kandi burruss explod rape accus real housew at...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com
1,peopl choic award best red carpet look,https://www.today.com/style/see-people-s-choic...,www.today.com
2,sophia bush send sweet birthday messag one tre...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com
3,colombian singer maluma spark rumour inappropr...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk
4,gossip girl year later upper east sider shock ...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com
...,...,...,...
23191,pippa middleton wed case miss pippa marri lace...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk
23192,zayn malik gigi hadid shock split chanc reunit...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com
23193,jessica chastain recal moment mother boyfriend...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com
23194,tristan thompson feel dump khlo kardashian ref...,www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com


In [ ]:
print(X.shape,Y.shape)

(22866, 3) (22866,)


In [ ]:
vectorizer=TfidfVectorizer()

In [ ]:
X_transformed = vectorizer.fit_transform(df['title'])

In [ ]:
X_transformed

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 180699 stored elements and shape (22866, 12472)>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, Y, test_size=0.2, stratify=Y, random_state=2)


In [ ]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (18292, 12472)
Shape of X_test: (4574, 12472)
Shape of y_train: (18292,)
Shape of y_test: (4574,)


In [ ]:
# training the model
model=LogisticRegression()

In [ ]:
model.fit(X_train,y_train)

LogisticRegression()

In [ ]:
# accuracy score


In [ ]:
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,y_train)

In [ ]:
print(training_data_accuracy)

0.8592827465558714


In [ ]:
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,y_test)

In [ ]:
print(test_data_accuracy)

0.8318758198513336


In [ ]:
X_new=X_test[0]

In [ ]:
Prediction=model.predict(X_new)
print(Prediction)

[1]


In [ ]:
print(y_test.iloc[0])

1
